In [1]:
import os
os.chdir('..')

%load_ext autoreload
%autoreload 2

In [3]:
import torch

import torch.nn as nn

from src.consts import (
    RANDOM_SEED, USE_CUDA, PROCESSED_DATA_DIR_NAME, BATCH_SIZE, MODEL_TYPE, EMBEDDINGS_SIZE, HIDDEN_UNIT_COUNT,
    LAYER_COUNT, DROPOUT_PROB, TIED, SEQUENCE_LENGTH, EVAL_BATCH_SIZE, INITIAL_LEARNING_RATE, EPOCHS, GRADIENT_CLIPPING,
    LOG_INTERVAL, MODEL_FILE_NAME
)
from src.corpus import Corpus
from src.model import RNNModel
from src.training import train

In [5]:
import time
start = time.time()
corpus = Corpus()
time.time() - start

292.36966156959534

In [6]:
print("{:,}".format(len(corpus.dictionary)))

602,755


In [4]:
# print("{:,}".format(len(corpus.dictionary)))

880,388


In [4]:
# print("{:,}".format(len(corpus.dictionary)))
# 624,399

624,399


In [4]:
# len(corpus.dictionary)
# 2,449,854

2449854

In [6]:
# import hashlib
# fn = 'models/corpus.{}.data'.format(hashlib.md5('data/interim/wiki_pt_splits'.encode()).hexdigest())

# torch.save(corpus, fn)

In [7]:
import time
start = time.time()
corpus.add_corpus_data()
time.time() - start

128.03656840324402

In [8]:
corpus.valid.size()

torch.Size([11606861])

In [9]:
device = torch.device("cuda")

In [10]:
ntokens = len(corpus.dictionary)
# model = RNNModel(MODEL_TYPE, ntokens, EMBEDDINGS_SIZE, HIDDEN_UNIT_COUNT, LAYER_COUNT, DROPOUT_PROB,
model = RNNModel(MODEL_TYPE, ntokens, EMBEDDINGS_SIZE, HIDDEN_UNIT_COUNT, 1, 0,
                 TIED).to(device)
criterion = nn.CrossEntropyLoss()

In [13]:
train(model, corpus, criterion, device)

RuntimeError: cuda runtime error (59) : device-side assert triggered at /pytorch/aten/src/THC/generic/THCTensorMath.cu:24

In [14]:
torch.cuda.max_memory_allocated(device=device)

2311067648

In [15]:
torch.cuda.memory_allocated(device=device)

623253504

In [16]:
model

RNNModel(
  (drop): Dropout(p=0)
  (encoder): Embedding(602755, 200)
  (rnn): LSTM(200, 200)
  (decoder): Linear(in_features=200, out_features=602755, bias=True)
)

In [17]:
for key, value in model.state_dict().items():
    print(key, value.size())

encoder.weight torch.Size([602755, 200])
rnn.weight_ih_l0 torch.Size([800, 200])
rnn.weight_hh_l0 torch.Size([800, 200])
rnn.bias_ih_l0 torch.Size([800])
rnn.bias_hh_l0 torch.Size([800])
decoder.weight torch.Size([602755, 200])
decoder.bias torch.Size([602755])


In [18]:
params = list(model.parameters()) + list(criterion.parameters())
total_params = sum(x.size()[0] * x.size()[1] if len(x.size()) > 1 else x.size()[0] for x in params if x.size())

In [21]:
print("{:,}".format(total_params))

121,475,355
